In [9]:
import os

def load(path):
    ham_list = list()
    spam_list = list()
    ham = path+'/ham'
    spam = path+'/spam'

    for f in os.listdir(ham):
        f = open(os.path.join(ham, f))
        ham_list.append(f.read().split())
        f.close()
    for f in os.listdir(spam):
        f = open(os.path.join(spam, f))
        spam_list.append(f.read().split())
        f.close()    
    return ham_list, spam_list

ham_list = list()
spam_list = list()
for i in range(1,6):
    ham, spam = load('data/enron'+str(i))
    ham_list.extend(ham)
    spam_list.extend(spam)
    

In [10]:
from collections import defaultdict

def train(ham_list, spam_list):
    nh = len(ham_list)
    ns = len(spam_list)

    prior = {
        'ham' : float(nh)/(nh+ns),
        'spam': float(ns)/(nh+ns)
    }
    condprob = {
        'ham' : defaultdict(float),
        'spam' : defaultdict(float)
    }
    
    v = {
        'ham' : defaultdict(int),
        'spam' : defaultdict(int)
    }
    voc = set()
    ch = 0
    cs = 0
    for text in ham_list:
        voc.update(text)
        for word in text:
            v['ham'][word] += 1
            ch += 1
    for text in spam_list:
        voc.update(text)
        for word in text:
            v['spam'][word] += 1
            cs += 1
    word_count = len(voc)
    
    for word in voc:
        count = v['ham'][word]
        condprob['ham'][word] = float(count+1)/(ch+word_count)
        #condprob['ham'][word] = float(count)/(ch)
    for word in voc:
        count = v['spam'][word]
        condprob['spam'][word] = float(count+1)/(cs+word_count)
        #condprob['spam'][word] = float(count)/(cs)
    return prior, condprob, voc

prior, condprob, voc = train(ham_list, spam_list)
#prior, condprob, voc = train([['Chinese', 'Beijing', 'Chinese'],
#                              ['Chinese', 'Chinese', 'Shanghai'],
#                              ['Chinese', 'Macao']], 
#                             [['Tokyo', 'Japan', 'Chinese']])
#prior, condprob 

In [11]:
import math
def test(data, prior, condprob, voc):
    score_ham = math.log(prior['ham'])
    score_spam = math.log(prior['spam'])
    for word in data:
        if word in voc:
            score_ham += math.log(condprob['ham'][word])
            score_spam += math.log(condprob['spam'][word])

    return max((score_ham, False), (score_spam, True))

test_ham, test_spam = load('data/enron6')
r = n = p = 0
for ham in test_ham:
    _, result = test(ham, prior, condprob, voc)
    if result:
        p += 1
    else:
        r += 1
        
for spam in test_spam:
    _, result = test(spam, prior, condprob, voc)
    if result:
        r += 1
    else:
        n += 1

In [12]:
r, n, p

(5889, 42, 69)

In [13]:
len(voc)

137152